# TF2 Multi Variable Regression

### Multi-Variable Linear Regression
* 입력 변수가 1개가 아닌 여러개인 경우 다변수 선형회귀
    * bias 도 입력 변수가 1인 것으로 생각할 수 있다
    * $y = W_0X_0 + W_1X_1 + W_2X_2$
        * $W_0$ : 상수, bias
        * $X_0$ : 1
        * $W_n$ : n번째 계수
        * $X_n$ : n번째 입력 변수
![image.png](https://i.imgur.com/ttVB6jg.png)

### 행렬곱
* 각각의 입력 변수에 대해서 미분을 하기 불편하므로 행렬 연산을 한다.
* 행렬곱 연산은 각각의 계수를 개별적으로 찾는 것을 쉽게 해준다.
* 이 때 전치행렬(Transpose)가 쓸모 있다.
![image.png](https://i.imgur.com/netr7gv.png)

### 행렬곱을 이용한 연산
* $H(X) = WX + b$
* $\begin{bmatrix} w1 & w2 & w3 \end{bmatrix} \times \begin{bmatrix} x1 \\ x2 \\x3\end{bmatrix} = 
\begin{bmatrix} w1 \times x1 + w2 \times x2 + w3 \times x3 \end{bmatrix}$

### bias 제거 단순화
* $H(X) = WX$
*  $\begin{bmatrix} b & w1 & w2 & w3 \end{bmatrix} \times \begin{bmatrix} 1\\ x1 \\ x2 \\x3\end{bmatrix} = 
\begin{bmatrix}b \times 1 +  w1 \times x1 + w2 \times x2 + w3 \times x3 \end{bmatrix}$

### 전치 행렬
* Transpose
![image.png](https://i.imgur.com/0MClWDZ.png)

### 전치행렬을 이용한 연산
* $W = \begin{bmatrix} b\\w1\\w2\\w3\end{bmatrix}, x = \begin{bmatrix} 1 \\x1\\x2\\x3\end{bmatrix}$ 일때
* $H(X) = W^TX$
* $ =  \begin{bmatrix}b & w1 & w2 & w3\end{bmatrix} 
 \times \begin{bmatrix} 1 \\ x1 \\ x2 \\ x3 \end{bmatrix} = 
\begin{bmatrix}b \times1 + w1 \times x1 + w2 \times x2 + w3 \times x3 \end{bmatrix}
$


### 2과목 점수 계산 예제
* 2과목의 시험 점수로 학점을 계산하는 예제를 만들어 보자.
    * 3개의 변수를 사용한다.
        * $x_0$ = 1, bias, 50점 기본점수
        * $x_1$ = 0~50, 가중치 0.7
        * $x_2$ = 0~50, 가중치 0.3
        * $y = 50x_0 + 0.7x_1 + 0.3x_2$

In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np

In [6]:
m = 50
TRUE_W1, TRUE_W2, TRUE_b = 0.7, 0.3, 50

X1 = np.random.randint(0, 50, (m,1)).astype(np.float32)
X2 = np.random.randint(0, 50, (m,1)).astype(np.float32)

Y = TRUE_W1 * X1 + TRUE_W2 * X2 + TRUE_b

print(np.hstack((X1,X2, Y))[:5])

[[16.  40.  73.2]
 [ 3.  15.  56.6]
 [ 5.   6.  55.3]
 [33.   8.  75.5]
 [ 3.   6.  53.9]]


### 개별 변수로 구현
* 개별 변수 선언

```
x1 = x[:,0]
x2 = x[:,1]

W1 = tf.Variable(tf.random.normal([1]))
W2 = tf.Variable(tf.random.normal([1]))
b = tf.Variable(tf.random.normal([1]))
```

* ` hypothesis = W1 * x1 + W2 * x2 + b`

In [ ]:
W1 = tf.Variable(tf.random.normal((1,)))
W2 = tf.Variable(tf.random.normal((1,)))
b = tf.Variable(tf.random.normal((1,)))
print(W1, W2, b)

learning_rate = 0.0005
epochs = 40001
for epoch in range(epochs):
    with tf.GradientTape() as tape:
        h = W1 * X1 + W2 * X2 + b
        cost = tf.reduce_mean(tf.square(h-Y))
    gW1, gW2, gb = tape.gradient(cost, [W1, W2, b])
    
    W1.assign_sub(learning_rate * gW1)
    W2.assign_sub(learning_rate * gW2)
    b.assign_sub(learning_rate * gb)
    
    if epoch % 2000 == 0:
        print(f"epoch:{epoch}, cost:{cost.numpy()}")
print(f"Final W1:{W1.numpy()}, W2:{W2.numpy()}, b:{b.numpy()}")

### Matrix로 구현
* `W = tf.Variable(tf.random.normal([2,1]))`
* `hypothesis = tf.matmul(x, W) + b`

In [4]:
a = np.arange(12, dtype=np.int32).reshape(4,3)
b = tf.constant([[2,3],
                 [2,3],
                 [2,3]])
print(a)
print(b)
print(tf.matmul(a, b))

[[ 0  1  2]
 [ 3  4  5]
 [ 6  7  8]
 [ 9 10 11]]
tf.Tensor(
[[2 3]
 [2 3]
 [2 3]], shape=(3, 2), dtype=int32)
tf.Tensor(
[[ 6  9]
 [24 36]
 [42 63]
 [60 90]], shape=(4, 2), dtype=int32)


In [ ]:
X = np.hstack((X1, X2))
print(X[:5])
print(X.shape)

W = tf.Variable(tf.random.normal([2,1]))
b = tf.Variable(tf.random.normal((1, )))
lr = 0.0005
epochs = 40001

for epoch in range(epochs):
    with tf.GradientTape() as tape:
        h = tf.matmul(X, W) + b
        cost = tf.reduce_mean(tf.square(h-Y))
    dW, db = tape.gradient(cost, [W, b])
    W.assign_sub(lr * dW)
    b.assign_sub(lr * db)
    
    if epoch % 2000 == 0:
        print(f"epoch:{epoch}, cost:{cost.numpy()}")
print(f"Final W:{W.numpy()}, b:{b.numpy()}")

[[16. 40.]
 [ 3. 15.]
 [ 5.  6.]
 [33.  8.]
 [ 3.  6.]]
(50, 2)
epoch:0, cost:1623.125
epoch:2000, cost:225.02149963378906
epoch:4000, cost:107.1850357055664
epoch:6000, cost:51.055503845214844
epoch:8000, cost:24.319345474243164
epoch:10000, cost:11.584070205688477
epoch:12000, cost:5.5178351402282715
epoch:14000, cost:2.6283254623413086
epoch:16000, cost:1.251959204673767
epoch:20000, cost:0.28407156467437744
epoch:22000, cost:0.13530224561691284
epoch:24000, cost:0.06445538252592087
epoch:26000, cost:0.0307094007730484
epoch:28000, cost:0.014626524411141872
epoch:30000, cost:0.006965349428355694
epoch:32000, cost:0.0033119800500571728
epoch:34000, cost:0.0015758691588416696
epoch:36000, cost:0.0007655743393115699
epoch:38000, cost:0.00037090160185471177
epoch:40000, cost:0.0001686997275101021
Final W:[[0.7006307]
 [0.3004512]], b:[49.96985]
